http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

no freezing of bert model

# start

In [1]:
PRFX='b0326_3'

In [2]:
trntmstmp=1584412344
valtmstmp=1585069785

import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


SEED=101

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


In [3]:
!nvidia-smi

Fri Mar 27 03:29:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   65C    P0    73W / 149W |      0MiB / 11441MiB |     95%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out = f'{HOME}/output/{PRFX}'
from pathlib import Path
Path(p_out).mkdir(exist_ok=True)

import torch
from transformers import *
import torch
device=torch.device('cuda')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score

pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)

import random
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(SEED)

In [5]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [6]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e6
)

CPU times: user 7.76 s, sys: 637 ms, total: 8.39 s
Wall time: 8.41 s


In [7]:
# lens = dftrn.toks.apply(lambda x: len(x.split('\t')))
# lens.mean(), np.percentile(lens, 50), np.percentile(lens, 95), np.percentile(lens, 99)
# (46.754583, 41.0, 106.0, 135.0)


# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=128
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [8]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    if not istrn: return X
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,ys
    

In [9]:
X,ys = mk_tensors(dftrn)

In [10]:
# BS=2
# ds = TensorDataset(X,ys)
# dl = DataLoader(ds, batch_size=BS, shuffle=True)
# for step, batch in enumerate(dl):
#     X_b,ys_b = (o.to(device) for o in batch)
#     print(X_b.shape,ys_b.shape)
#     break
# torch.Size([2, 128]) torch.Size([2, 4])

# model

In [11]:
BS=8
EPOCHS=4
WD=0.01
LR=1e-5
SCHDLR_FUNC = get_cosine_schedule_with_warmup 
WARMUP_RATE = 0.05
N_CYCLS = .5
EPS = 1e-6

In [12]:
# ds = TensorDataset(X,ys)
# dl = DataLoader(ds, batch_size=BS, shuffle=True)
# for step, batch in enumerate(dl):
#     X_b,ys_b = (o.to(device) for o in batch)
#     print(X_b.shape,ys_b.shape)
#     break

# # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
# bertmodel=bertmodel.eval()
# bertmodel.to(device)
# with torch.no_grad():
#     last_hidden_state, pooler_output, hidden_states = bertmodel(X_b,)
#     avg_pool = torch.mean(last_hidden_state,1)
#     max_pool,_ = torch.max(last_hidden_state,1)
# last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape
# (torch.Size([2, 128, 768]),
#  torch.Size([2, 768]),
#  13,
#  torch.Size([2, 768]),
#  torch.Size([2, 768]))

In [13]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel        
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN*2
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, x):
        attn_msk = (x!=tokenizer.pad_token_id).float().to(device)
        segments = torch.zeros(x.shape, dtype=torch.long).to(device)
        bert_output=self.bertmodel(x,attention_mask=attn_msk,token_type_ids=segments)
        last_hidden_state,pooler_output,hidden_states = bert_output
        avg_pool = torch.mean(last_hidden_state,1)
        max_pool,_ = torch.max(last_hidden_state,1)
        x = torch.cat([avg_pool,max_pool],1) 
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

# train

In [14]:
msk_vl = np.random.rand(len(X))<0.15
tr = np.where(~msk_vl)[0]
vl = np.where( msk_vl)[0]
Xtr,Xvl = X[tr],X[vl]
ys_tr, ys_vl = ys[tr], ys[vl]
ds_tr = TensorDataset(Xtr,ys_tr)
ds_vl = TensorDataset(Xvl,ys_vl)
dl_tr = DataLoader(ds_tr, batch_size=BS,   pin_memory=True, shuffle=True)
dl_vl = DataLoader(ds_vl, batch_size=BS, pin_memory=True, shuffle=False)

In [15]:
model = TwtModel(bertmodel)
model = model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WD},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=EPS)
t_total = int(EPOCHS*len(dl_tr))
scheduler = SCHDLR_FUNC(
    optimizer, 
    num_warmup_steps=int(WARMUP_RATE*t_total), 
    num_training_steps=t_total,
    num_cycles=N_CYCLS,
)

In [16]:
def evaluate(model, dl):
    prd = []
    ys = []
    losses = []
    model.eval()
    for step, dat_b in enumerate(dl):
        model.eval()
        x_b,ys_b = (o.to(device) for o in dat_b)
        prd_b = model(x_b)

        ys.append(ys_b.cpu().detach().numpy())
        prd.append(prd_b.cpu().detach().numpy())

        loss_ =  F.binary_cross_entropy_with_logits(prd_b,ys_b)
        losses.append(loss_.item())

    prd = np.concatenate(prd)
    ys = np.concatenate(ys)
    scrs = [roc_auc_score(ys[:,i], prd[:,i]) for i in range(4)]
    scr  = roc_auc_score(ys, prd)
    loss = np.mean(losses)
    return prd, ys, scrs, scr, loss

In [17]:
model.zero_grad()
set_seed(SEED)

epc2loss_tr = []
epc2loss_vl = []
epc2scrs_tr = []
epc2scrs_vl = []
best_scr = float('-inf')
save_p = f'{p_out}/mdl.p'
for epoch in range(EPOCHS):
    print(dtnow(), f'epoch {epoch} starts')
    ys_tr_ep = []
    prd_tr_ep = []
    for step, dat_b in enumerate(dl_tr):
        model.train()
        x_b,ys_b = (o.to(device) for o in dat_b)
        prd_b = model(x_b)
        loss =  F.binary_cross_entropy_with_logits(prd_b,ys_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

        ys_tr_ep.append(ys_b.cpu().detach().numpy())
        prd_tr_ep.append(prd_b.cpu().detach().numpy())

    prd_tr_ep = np.concatenate(prd_tr_ep)
    ys_tr_ep = np.concatenate(ys_tr_ep)
    scrs_tr = [roc_auc_score(ys_tr_ep[:,i], prd_tr_ep[:,i]) for i in range(4)]
    scr_tr  = roc_auc_score(ys_tr_ep, prd_tr_ep)
    loss_tr = loss.item()
    
    prd_vl_ep,ys_vl_ep,scrs_vl,scr_vl,loss_vl = evaluate(model, dl_vl)
    print(f'lss_tr: {loss_tr: .4f}; scr_tr: {scr_tr: .4f}; {scrs_tr}')
    print(f'lss_vl: {loss_vl: .4f}; scr_vl: {scr_vl: .4f}; {scrs_vl}')
    if scr_vl>best_scr: 
        best_scr = scr_vl
        best_prd_vl = prd_vl_ep
        print(f'found better scr, saving model...')
        torch.save(model.state_dict(),save_p)
    
    epc2loss_tr.append(loss_tr)
    epc2loss_vl.append(loss_vl)
    epc2scrs_tr.append(scrs_tr)
    epc2scrs_vl.append(scrs_vl)


2020-03-27 03:30:28 epoch 0 starts


KeyboardInterrupt: 

In [19]:
step, len(dl_tr)

(73985, 106285)